In [62]:
from azureml.opendatasets import NycTlcYellow
from datetime import datetime
from dateutil import parser

start_date = parser.parse('2018-05-01')
end_date = parser.parse('2018-05-07')
nyc_tlc = NycTlcYellow(start_date=start_date, end_date=end_date)
nyc_tlc_df = nyc_tlc.to_pandas_dataframe()
nyc_tlc_df.info()

StatementMeta(sparkpool, 9, 63, Finished, Available)

[Info] read from /tmp/tmp9km9zxf0/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00000-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426339-118.c000.snappy.parquet
[Info] read from /tmp/tmp9km9zxf0/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00001-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426336-117.c000.snappy.parquet
[Info] read from /tmp/tmp9km9zxf0/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00002-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426334-119.c000.snappy.parquet
[Info] read from /tmp/tmp9km9zxf0/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00003-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426340-115.c000.snappy.parquet
[Info] read from /tmp/tmp9km9zxf0/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/yellow/puYear=2018/puMonth=5/part-00004-ti

In [63]:
from IPython.display import display

sampled_df = nyc_tlc_df.sample(n=10000, random_state=123)
display(sampled_df.head(5))

StatementMeta(sparkpool, 9, 64, Finished, Available)

,vendorID,tpepPickupDateTime,tpepDropoffDateTime,passengerCount,tripDistance,puLocationId,doLocationId,startLon,startLat,endLon,...,rateCodeId,storeAndFwdFlag,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,totalAmount
87213,2,2018-05-05 19:07:01,2018-05-05 19:28:44,1,3.95,164,112,NaN,NaN,NaN,...,1,N,2,17.0,0.0,0.5,0.3,0.00,5.76,23.56
145405,2,2018-05-05 22:46:06,2018-05-05 22:59:11,1,1.22,264,264,NaN,NaN,NaN,...,1,N,1,9.5,0.5,0.5,0.3,2.16,0.00,12.96
457648,1,2018-05-06 18:53:06,2018-05-06 19:06:31,1,2.20,246,162,NaN,NaN,NaN,...,1,N,2,11.0,0.0,0.5,0.3,0.00,0.00,11.80
369051,2,2018-05-02 09:25:13,2018-05-02 09:36:32,1,0.89,161,162,NaN,NaN,NaN,...,1,N,1,8.0,0.0,0.5,0.3,1.76,0.00,10.56
38871,2,2018-05-04 02:58:10,2018-05-04 03:01:10,3,0.45,79,4,NaN,NaN,NaN,...,1,N,1,4.0,0.5,0.5,0.3,1.32,0.00,6.62


In [64]:
import numpy
import pandas

def get_pickup_time(df):
    pickupHour = df['pickupHour'];
    if ((pickupHour >= 7) & (pickupHour <= 10)):
        return '1'
    elif ((pickupHour >= 11) & (pickupHour <= 15)):
        return '2'
    elif ((pickupHour >= 16) & (pickupHour <= 19)):
        return '3'
    else:
        return '4'

featurized_df = pandas.DataFrame()
featurized_df['tipped'] = (sampled_df['tipAmount'] > 0).astype('int')
featurized_df['fareAmount'] = sampled_df['fareAmount'].astype('float32')
featurized_df['paymentType'] = sampled_df['paymentType'].astype('int')
featurized_df['passengerCount'] = sampled_df['passengerCount'].astype('int')
featurized_df['tripDistance'] = sampled_df['tripDistance'].astype('float32')
featurized_df['pickupHour'] = sampled_df['tpepPickupDateTime'].dt.hour.astype('int')
featurized_df['tripTimeSecs'] = ((sampled_df['tpepDropoffDateTime'] - sampled_df['tpepPickupDateTime']) / numpy.timedelta64(1, 's')).astype('int')

featurized_df['pickupTimeBin'] = featurized_df.apply(get_pickup_time, axis=1)
featurized_df = featurized_df.drop(columns='pickupHour')

display(featurized_df.head(5))

StatementMeta(sparkpool, 9, 65, Finished, Available)

,tipped,fareAmount,paymentType,passengerCount,tripDistance,tripTimeSecs,pickupTimeBin
87213,0,17.0,2,1,3.95,1303,3
145405,1,9.5,1,1,1.22,785,4
457648,0,11.0,2,1,2.20,805,3
369051,1,8.0,1,1,0.89,679,1
38871,1,4.0,1,3,0.45,180,4


In [65]:
filtered_df = featurized_df[(featurized_df.tipped >= 0) & (featurized_df.tipped <= 1)\
    & (featurized_df.fareAmount >= 1) & (featurized_df.fareAmount <= 250)\
    & (featurized_df.paymentType >= 1) & (featurized_df.paymentType <= 2)\
    & (featurized_df.passengerCount > 0) & (featurized_df.passengerCount < 8)\
    & (featurized_df.tripDistance >= 0) & (featurized_df.tripDistance <= 100)\
    & (featurized_df.tripTimeSecs >= 30) & (featurized_df.tripTimeSecs <= 7200)]

filtered_df.info()

StatementMeta(sparkpool, 9, 66, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9776 entries, 87213 to 333274
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tipped          9776 non-null   int64  
 1   fareAmount      9776 non-null   float32
 2   paymentType     9776 non-null   int64  
 3   passengerCount  9776 non-null   int64  
 4   tripDistance    9776 non-null   float32
 5   tripTimeSecs    9776 non-null   int64  
 6   pickupTimeBin   9776 non-null   object 
dtypes: float32(2), int64(4), object(1)
memory usage: 534.6+ KB


In [66]:
display(filtered_df)

StatementMeta(sparkpool, 9, 67, Finished, Available)

,tipped,fareAmount,paymentType,passengerCount,tripDistance,tripTimeSecs,pickupTimeBin
87213,0,17.0,2,1,3.95,1303,3
145405,1,9.5,1,1,1.22,785,4
457648,0,11.0,2,1,2.20,805,3
369051,1,8.0,1,1,0.89,679,1
38871,1,4.0,1,3,0.45,180,4
...,...,...,...,...,...,...,...
252911,1,12.0,1,1,3.00,821,4
69465,0,20.0,2,1,6.10,1932,1
117339,1,5.0,1,1,0.64,295,3
355108,0,26.0,1,1,9.22,800,1


In [67]:
  train_df = filtered_df[["tipped","paymentType","passengerCount","tripDistance","tripTimeSecs","pickupTimeBin","fareAmount"]]

StatementMeta(sparkpool, 9, 68, Finished, Available)

In [68]:
import os
import shutil
import mlflow
import json
from mlflow.utils import model_utils

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression


class LinearRegressionModel():
  _ARGS_FILENAME = 'args.json'
  FEATURES_KEY = 'features'
  TARGETS_KEY = 'targets'
  TARGETS_PRED_KEY = 'targets_pred'

  def __init__(self, fit_intercept, nb_input_features=9, nb_output_features=1):
    self.fit_intercept = fit_intercept
    self.nb_input_features = nb_input_features
    self.nb_output_features = nb_output_features

  def get_args(self):
    args = {
        'nb_input_features': self.nb_input_features,
        'nb_output_features': self.nb_output_features,
        'fit_intercept': self.fit_intercept
    }
    return args

  def create_model(self):
    self.model = LinearRegression(fit_intercept=self.fit_intercept)

  def train(self, dataset):

    features = np.stack([sample for sample in iter(
        dataset[LinearRegressionModel.FEATURES_KEY])], axis=0)

    targets = np.stack([sample for sample in iter(
        dataset[LinearRegressionModel.TARGETS_KEY])], axis=0)


    self.model.fit(features, targets)

  def predict(self, dataset):
    features = np.stack([sample for sample in iter(
        dataset[LinearRegressionModel.FEATURES_KEY])], axis=0)
    targets_pred = self.model.predict(features)
    return targets_pred

  def save(self, path):
    if os.path.exists(path):
      shutil.rmtree(path)

    # save the sklearn model with mlflow
    mlflow.sklearn.save_model(self.model, path)

    # save args
    self._save_args(path)

  def _save_args(self, path):
    args_filename = os.path.join(path, LinearRegressionModel._ARGS_FILENAME)
    with open(args_filename, 'w') as f:
      args = self.get_args()
      json.dump(args, f)


def train(train_df, output_model_path):
  print(f"Start to train LinearRegressionModel.")

  # Initialize input dataset
  dataset = train_df.to_numpy()
  datasets = {}
  datasets['targets'] = dataset[:, -1]
  datasets['features'] = dataset[:, :6]

  # Initialize model class obj
  model_class = LinearRegressionModel(fit_intercept=10)
  with mlflow.start_run(nested=True) as run:
    model_class.create_model()
    model_class.train(datasets)
    model_class.save(output_model_path)
    print(model_class.predict(datasets))


train(train_df, './artifacts/output')

StatementMeta(sparkpool, 9, 69, Finished, Available)

Start to train LinearRegressionModel.
[17.0219659   9.05096899 10.97521973 ...  5.43188232 25.43395136
 16.50693599]


In [69]:
from azureml.core import Workspace, Model
from azureml.core.authentication import ServicePrincipalAuthentication
from notebookutils.mssparkutils import azureML

ws = azureML.getWorkspace("<linked_service_name>")

model = Model.register(
    model_path="./artifacts/output",
    model_name="mlflow_sklearn_nyctaxi",
    workspace=ws,
)



StatementMeta(sparkpool, 9, 70, Finished, Available)

Registering model mlflow_sklearn_nyctaxi


In [70]:
from pyspark.sql.functions import col, pandas_udf,udf,lit
import azure.synapse.ml.predict as pcontext

StatementMeta(sparkpool, 9, 71, Finished, Available)

In [74]:
DATA_FILE = "abfss://<containername>@<storageaccountname>.dfs.core.windows.net/test_data.csv"
AML_MODEL_URI_SKLEARN = "aml://mlflow_sklearn_nyctaxi" #You can add a version by doing "aml://mlflow_sklearn_nyctaxi:1" ":1" is the version of model in AML. We can choose which version we want to run. If ":1" is not provided then by default latest version will be picked
RETURN_TYPES = "FLOAT"

StatementMeta(sparkpool, 9, 75, Finished, Available)

In [72]:
spark.conf.set("spark.synapse.ml.predict.enabled","true")

StatementMeta(sparkpool, 9, 73, Finished, Available)

In [75]:
model = pcontext.bind_model(
    RETURN_TYPES, 
    "mlflow",
    "sklearn_linear_regression",
    AML_MODEL_URI_SKLEARN,
    aml_workspace=ws
    ).register()

StatementMeta(sparkpool, 9, 76, Finished, Available)

In [76]:
df = spark.read.format("csv").option("header", "true").csv(DATA_FILE,
        inferSchema=True)


StatementMeta(sparkpool, 9, 77, Finished, Available)

In [77]:
df=df.replace('AMRush',"1").replace('Afternoon',"2").replace('PMRush',"3").replace('Night',"4")
df=df.withColumn("pickupTimeBin",df.pickupTimeBin.cast('int'))
df = df.drop('fareAmount')

StatementMeta(sparkpool, 9, 78, Finished, Available)

In [78]:

df.createOrReplaceTempView('data')
df.show(10)
df

StatementMeta(sparkpool, 9, 79, Finished, Available)

+------+-----------+--------------+------------+------------+-------------+
|tipped|paymentType|passengerCount|tripDistance|tripTimeSecs|pickupTimeBin|
+------+-----------+--------------+------------+------------+-------------+
|     0|          2|             1|         3.1|         547|            4|
|     0|          2|             1|         5.7|        1080|            4|
|     1|          1|             3|         0.7|         240|            2|
|     1|          1|             1|         9.5|        1973|            4|
|     0|          2|             1|         1.7|         767|            2|
|     1|          1|             2|         4.8|        1442|            4|
|     1|          1|             6|        8.41|        3110|            3|
|     1|          1|             1|         1.5|         545|            4|
|     1|          1|             1|        4.11|        1679|            2|
|     1|          1|             1|         0.6|         283|            3|
+------+----

DataFrame[tipped: int, paymentType: int, passengerCount: int, tripDistance: double, tripTimeSecs: int, pickupTimeBin: int]

In [79]:
spark.sql(
    """
        select * from data
    """
).show()


StatementMeta(sparkpool, 9, 80, Finished, Available)

+------+-----------+--------------+------------+------------+-------------+
|tipped|paymentType|passengerCount|tripDistance|tripTimeSecs|pickupTimeBin|
+------+-----------+--------------+------------+------------+-------------+
|     0|          2|             1|         3.1|         547|            4|
|     0|          2|             1|         5.7|        1080|            4|
|     1|          1|             3|         0.7|         240|            2|
|     1|          1|             1|         9.5|        1973|            4|
|     0|          2|             1|         1.7|         767|            2|
|     1|          1|             2|         4.8|        1442|            4|
|     1|          1|             6|        8.41|        3110|            3|
|     1|          1|             1|         1.5|         545|            4|
|     1|          1|             1|        4.11|        1679|            2|
|     1|          1|             1|         0.6|         283|            3|
|     1|    

In [80]:
predictions = spark.sql(
                  """
                      SELECT PREDICT('sklearn_linear_regression', *) AS predict FROM data
                  """
              ).show()

StatementMeta(sparkpool, 9, 81, Finished, Available)

+---------+
|  predict|
+---------+
|11.309419|
| 19.21058|
| 5.265986|
|31.547586|
| 9.878617|
|19.487162|
|35.235844|
| 8.345654|
|19.582026|
|5.2904415|
| 7.024831|
|10.039314|
| 8.098974|
| 3.859315|
| 6.632982|
| 6.966037|
| 9.597531|
|9.2682295|
|4.8557224|
|17.350607|
+---------+
only showing top 20 rows

